In [ ]:
'''Dataset Creation :
- numpy for random number generation
- pandas for dataset creation and manipulation
- datetime/timedelta for working with dates
- Fix the random seed (42) so results are reproducible.
- Define a fixed "today" date (Sept 20, 2025). This ensures our date calculations are consistent.
- I've simulate 5000 users. (Created unique IDs from 1 to 5000.)
- For each user, randomly assign a last_order_date between 1 and 365 days ago.
- For 20% of users → assign a random abandonment date within the last 14 days.
- For the remaining 80% → mark as NaT (missing), meaning no abandonment.

Avg Order value Logic
- Most customers have low spend, but a few spend a lot.
- Cap values at 15,000 to avoid unrealistic outliers.
- num_cart_items: Random integer between 1 and 10 items.

- Define the universe of interest: only users who abandoned a cart within the last 7 days.


In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

np.random.seed(42)   # reproducibility of data
TODAY = datetime(2025, 9, 20)

# Number of fake users
N_USERS = 5000

# user IDs
user_id = np.arange(1, N_USERS+1)

# last order date (somewhere in past 365 days)
last_order_date = [TODAY - timedelta(days=int(x)) for x in np.random.randint(1, 365, size=N_USERS)]

# cart abandoned date: about 20% in last 14 days, rest missing
cart_abandoned_date = []
for i in range(N_USERS):
    if np.random.rand() < 0.2:  # 20% abandoned
        days_ago = np.random.randint(0, 14)
        cart_abandoned_date.append(TODAY - timedelta(days=days_ago))
    else:
        cart_abandoned_date.append(pd.NaT)

# avg order value (exponential distribution, capped)
avg_order_value = np.round(np.random.exponential(scale=800, size=N_USERS) + 50)
avg_order_value = np.minimum(avg_order_value, 15000)

# sessions, cart items
sessions_last_30d = np.random.poisson(4, size=N_USERS)
num_cart_items = np.random.randint(1, 10, size=N_USERS)

# engagement + profitability scores
engagement_score = np.clip(np.random.beta(2, 3, size=N_USERS), 0, 1)
profitability_score = np.clip(np.random.beta(2, 2, size=N_USERS), 0, 1)

# build DataFrame
df = pd.DataFrame({
    "user_id": user_id,
    "last_order_date": last_order_date,
    "cart_abandoned_date": cart_abandoned_date,
    "avg_order_value": avg_order_value,
    "sessions_last_30d": sessions_last_30d,
    "num_cart_items": num_cart_items,
    "engagement_score": engagement_score,
    "profitability_score": profitability_score
})

# derive days_since_abandon
df['days_since_abandon'] = (TODAY - pd.to_datetime(df['cart_abandoned_date'])).dt.days

# Universe: abandoned in last 7 days
universe = df[df['cart_abandoned_date'].notna() & (df['days_since_abandon'] <= 7)].copy()

print("Total dataset size:", len(df))
print("Universe size (7d abandoners):", len(universe))
print(universe.head())

# save
df.to_csv("mock_cart_data.csv", index=False)
universe.to_csv("mock_universe_7d.csv", index=False)


Total dataset size: 5000
Universe size (7d abandoners): 590
    user_id last_order_date cart_abandoned_date  avg_order_value  \
4         5      2025-07-10          2025-09-15            300.0   
10       11      2024-10-24          2025-09-14            819.0   
27       28      2025-01-27          2025-09-19           2208.0   
33       34      2024-12-23          2025-09-19           1223.0   
34       35      2025-03-14          2025-09-14            418.0   

    sessions_last_30d  num_cart_items  engagement_score  profitability_score  \
4                   3               7          0.296136             0.838315   
10                  2               8          0.299338             0.597137   
27                  7               4          0.674150             0.500903   
33                  3               7          0.237269             0.695012   
34                  4               9          0.199722             0.357460   

    days_since_abandon  
4                  5.0  


In [2]:
from google.colab import files

files.download('/content/mock_cart_data.csv')
files.download('/content/mock_universe_7d.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os


# Configs for dataset
# -------------------------
np.random.seed(42)
TODAY = pd.to_datetime("2025-09-20")
N_USERS = 5000

MIN_SEG = 100    # minimum desired segment size
MAX_SEG = 20000  # maximum desired segment size


# Data set creation
# -------------------------
def simulate_dataset(n_users=N_USERS, today=TODAY):
    user_id = np.arange(1, n_users+1)
    last_order_date = [today - timedelta(days=int(x)) for x in np.random.randint(1, 365, size=n_users)]

    cart_abandoned_date = []
    for _ in range(n_users):
        if np.random.rand() < 0.2:  # 20% have a recent abandonment (within 14 days)
            days_ago = np.random.randint(0, 14)
            cart_abandoned_date.append(today - timedelta(days=days_ago))
        else:
            cart_abandoned_date.append(pd.NaT)

    avg_order_value = np.round(np.random.exponential(scale=800, size=n_users) + 50)
    avg_order_value = np.minimum(avg_order_value, 15000)

    sessions_last_30d = np.random.poisson(4, size=n_users)
    num_cart_items = np.random.randint(1, 10, size=n_users)

    engagement_score = np.clip(np.random.beta(2, 3, size=n_users), 0, 1)
    profitability_score = np.clip(np.random.beta(2, 2, size=n_users), 0, 1)

    df = pd.DataFrame({
        "user_id": user_id,
        "last_order_date": last_order_date,
        "cart_abandoned_date": cart_abandoned_date,
        "avg_order_value": avg_order_value,
        "sessions_last_30d": sessions_last_30d,
        "num_cart_items": num_cart_items,
        "engagement_score": engagement_score,
        "profitability_score": profitability_score
    })
    df['days_since_abandon'] = (today - pd.to_datetime(df['cart_abandoned_date'])).dt.days
    return df


#  MECE segmentation function and our scoring logic.
# -------------------------
def build_mece_segments(df,
                        aov_thresholds=(1000, 3000),
                        eng_thresholds=(0.4, 0.7),
                        prof_threshold=0.6,
                        min_seg=MIN_SEG,
                        max_seg=MAX_SEG):
    df = df.copy()

    # AOV bucket
    lo, hi = aov_thresholds
    def aov_label(a):
        if pd.isna(a): return "AOV Other"
        if a > hi: return "High AOV"
        if a > lo: return "Mid AOV"
        return "Low AOV"
    df['aov_bucket'] = df['avg_order_value'].apply(aov_label)

    # Engagement buckets
    e_lo, e_hi = eng_thresholds
    def eng_label(e):
        if pd.isna(e): return "Eng Other"
        if e > e_hi: return "High Eng"
        if e > e_lo: return "Med Eng"
        return "Low Eng"
    df['eng_bucket'] = df['engagement_score'].apply(eng_label)

    # Profit buckets
    def prof_label(p):
        if pd.isna(p): return "Prof Other"
        if p > prof_threshold: return "High Prof"
        return "Low Prof"
    df['prof_bucket'] = df['profitability_score'].apply(prof_label)

    # Raw segment label (MECE Paramaters)
    df['segment_raw'] = df['aov_bucket'] + " | " + df['eng_bucket'] + " | " + df['prof_bucket']

    # Label mapping
    seg_counts = df['segment_raw'].value_counts().to_dict()
    final_map = {seg: seg for seg in seg_counts.keys()}

    # Helper to get parent label (one level up)
    def parent_of(seg, levels_up=1):
        parts = seg.split(" | ")
        if levels_up >= len(parts):
            return "ELSE"
        return " | ".join(parts[:-levels_up])

    # fold small segments up one level until sizes >= min_seg or become ELSE
    changed = True
    it = 0
    while changed and it < 10:
        it += 1
        changed = False
        # compute current mapped sizes
        mapped_sizes = {}
        for raw, mapped in final_map.items():
            mapped_sizes[mapped] = mapped_sizes.get(mapped, 0) + seg_counts.get(raw, 0)
        # folding small ones
        for raw, mapped in list(final_map.items()):
            size = mapped_sizes.get(mapped, 0)
            if size < min_seg:
                parent = parent_of(mapped, 1)
                if parent != mapped:
                    final_map[raw] = parent
                    changed = True

    # Apply mapping to Dataset we generagted
    df['final_segment'] = df['segment_raw'].map(final_map)

    # Split very-large final segments by engagement
    final_counts = df['final_segment'].value_counts()
    for seg_label, sz in final_counts.items():
        if sz > max_seg:
            mask = df['final_segment'] == seg_label
            # use rank to ensure deterministic behavior
            df.loc[mask, 'eng_rank'] = df.loc[mask, 'engagement_score'].rank(method='first')
            df.loc[mask, 'eng_tercile'] = pd.qcut(df.loc[mask, 'eng_rank'], 3, labels=['LowT','MidT','HighT'])
            for t in ['LowT','MidT','HighT']:
                df.loc[mask & (df['eng_tercile'] == t), 'final_segment'] = f"{seg_label} | Eng{t}"
            df = df.drop(columns=['eng_rank'], errors='ignore')

    return df


# Scoring function
# -------------------------
def score_segments(df):
    df = df.copy()

    # Conversion potential (engagement * recency factor)
    df['recency_factor'] = (7 - df['days_since_abandon'] + 1) / 8.0
    df['conversion_potential_ind'] = df['engagement_score'] * df['recency_factor']

    # Simulate lift vs control (seeded RNG for reproducibility)
    rng = np.random.default_rng(0)
    def sim_lift(row):
        base = 1.0 + 0.5 * (row['engagement_score'] - 0.5) + 0.4 * (row['profitability_score'] - 0.5)
        return float(np.clip(rng.normal(loc=base, scale=0.12), 0.5, 2.0))
    df['lift_ind'] = df.apply(sim_lift, axis=1)

    # Strategic fit: AOV percentile weighted by profitability
    df['aov_pct'] = df['avg_order_value'].rank(pct=True)
    df['strategic_fit_ind'] = 0.6 * df['aov_pct'] + 0.4 * df['profitability_score']

    # Aggregate metrics at final_segment level
    agg = df.groupby('final_segment').agg(
        size=('user_id','count'),
        conv_pot_mean=('conversion_potential_ind','mean'),
        lift_mean=('lift_ind','mean'),
        profit_mean=('profitability_score','mean'),
        strategic_mean=('strategic_fit_ind','mean'),
        avg_aov=('avg_order_value','mean')
    ).reset_index()

    # Normalized size (0-1)
    if agg['size'].max() > agg['size'].min():
        agg['size_norm'] = (agg['size'] - agg['size'].min()) / (agg['size'].max() - agg['size'].min())
    else:
        agg['size_norm'] = 0.0

    # Weighted overall score
    weights = {'conv_pot':0.32, 'lift':0.22, 'profitability':0.20, 'strategic':0.16, 'size_norm':0.10}
    agg['overall'] = (weights['conv_pot'] * agg['conv_pot_mean'] +
                      weights['lift'] * (agg['lift_mean'] / 2.0) +
                      weights['profitability'] * agg['profit_mean'] +
                      weights['strategic'] * agg['strategic_mean'] +
                      weights['size_norm'] * agg['size_norm'])

    agg['valid'] = agg['size'].between(MIN_SEG, MAX_SEG)
    return agg


# Agent WrkFlow
# -------------------------
# simulate data and save
df_all = simulate_dataset(N_USERS, TODAY)
df_all.to_csv("mock_cart_data.csv", index=False)

#build universe (abandoned in last 7 days)
universe = df_all[df_all['cart_abandoned_date'].notna() & (df_all['days_since_abandon'] >= 0) & (df_all['days_since_abandon'] <= 7)].copy()
universe.to_csv("mock_universe_7d.csv", index=False)

#build segments and score
segged = build_mece_segments(universe, min_seg=MIN_SEG, max_seg=MAX_SEG)
agg = score_segments(segged)
agg_sorted = agg.sort_values('overall', ascending=False).reset_index(drop=True)

#segmentd columns
def rule_from_label(label):
    parts = str(label).split(" | ")
    rules = []
    for p in parts:
        if p.startswith("High AOV"): rules.append("avg_order_value > 3000")
        elif p.startswith("Mid AOV"): rules.append("1000 < avg_order_value <= 3000")
        elif p.startswith("Low AOV"): rules.append("avg_order_value <= 1000")
        elif p.startswith("AOV Other"): rules.append("avg_order_value IS NULL")
        elif p.startswith("High Eng"): rules.append("engagement_score > 0.7")
        elif p.startswith("Med Eng"): rules.append("0.4 < engagement_score <= 0.7")
        elif p.startswith("Low Eng"): rules.append("engagement_score <= 0.4")
        elif p.startswith("Eng Other"): rules.append("engagement_score IS NULL")
        elif p.startswith("High Prof"): rules.append("profitability_score > 0.6")
        elif p.startswith("Low Prof"): rules.append("profitability_score <= 0.6")
        elif p.startswith("Prof Other"): rules.append("profitability_score IS NULL")
        elif p.startswith("ELSE"): rules.append("ELSE (merged)")
        else:
            rules.append(p)
    return " & ".join(rules)

agg_sorted['rule'] = agg_sorted['final_segment'].apply(rule_from_label)

#Summary and outputs of the scoring
agg_sorted.to_csv("segment_summary.csv", index=False)
segged.to_csv("segments_out_segmented.csv", index=False)
agg_sorted[['final_segment','rule','size','overall','valid']].to_csv("segment_rules.csv", index=False)
agg_sorted.to_json("segments_out.json", orient='records')

# Files that are storing the data.
print("Wrote files:")
print(" - mock_cart_data.csv (full simulated dataset)")
print(" - mock_universe_7d.csv (universe: last 7d abandoners)")
print(" - segment_summary.csv (aggregated segment scores)")
print(" - segments_out_segmented.csv (user-level with final_segment)")
print(" - segment_rules.csv (human-readable rules)")

print("\nTop segments (by overall score):")
display(agg_sorted.head(10))


Wrote files:
 - mock_cart_data.csv (full simulated dataset)
 - mock_universe_7d.csv (universe: last 7d abandoners)
 - segment_summary.csv (aggregated segment scores)
 - segments_out_segmented.csv (user-level with final_segment)
 - segment_rules.csv (human-readable rules)

Top segments (by overall score):


,final_segment,size,conv_pot_mean,lift_mean,profit_mean,strategic_mean,avg_aov,size_norm,overall,valid,rule
0,Mid AOV,185,0.237764,0.971548,0.498979,0.690778,1612.670270,1.0000,0.493275,True,1000 < avg_order_value <= 3000
1,Low AOV | Med Eng,162,0.306709,1.015822,0.538089,0.413923,455.771605,0.7125,0.454983,True,avg_order_value <= 1000 & 0.4 < engagement_sco...
2,ELSE,105,0.251007,1.053363,0.642414,0.506089,945.552381,0.0000,0.405649,True,ELSE (merged)
3,Low AOV | Low Eng | Low Prof,138,0.139087,0.822669,0.373452,0.352896,469.956522,0.4125,0.307405,True,avg_order_value <= 1000 & engagement_score <= ...


Lang chain

In [13]:
# !pip install -q langchain langchain-community openai ipywidgets pandas numpy requests

import os, json, traceback
from getpass import getpass
from datetime import datetime, timedelta
import numpy as np, pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output


# OpenRouter and Mistral for the LLM
# =====================
print("Paste your OpenRouter API key when prompted (press Enter to skip).")
openrouter_key = getpass("OpenRouter API key: ").strip()
if openrouter_key:
    os.environ["OPENAI_API_KEY"] = openrouter_key
    os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
    import openai
    openai.api_key = openrouter_key
    openai.api_base = "https://openrouter.ai/api/v1"


# Backend functions - Dataset creation and Scoring logic
# =====================
TODAY = pd.to_datetime("2025-09-20")
MIN_SEG, MAX_SEG = 100, 20000

def simulate_dataset(n_users=5000, today=TODAY):
    np.random.seed(42)
    user_id = np.arange(1, n_users+1)
    last_order_date = [today - timedelta(days=int(x)) for x in np.random.randint(1, 365, size=n_users)]
    cart_abandoned_date = []
    for _ in range(n_users):
        if np.random.rand() < 0.2:
            days_ago = np.random.randint(0, 14)
            cart_abandoned_date.append(today - timedelta(days=days_ago))
        else:
            cart_abandoned_date.append(pd.NaT)
    avg_order_value = np.round(np.random.exponential(scale=800, size=n_users) + 50)
    avg_order_value = np.minimum(avg_order_value, 15000)
    sessions_last_30d = np.random.poisson(4, size=n_users)
    num_cart_items = np.random.randint(1, 10, size=n_users)
    engagement_score = np.clip(np.random.beta(2, 3, size=n_users), 0, 1)
    profitability_score = np.clip(np.random.beta(2, 2, size=n_users), 0, 1)
    df = pd.DataFrame({
        "user_id": user_id,
        "last_order_date": last_order_date,
        "cart_abandoned_date": cart_abandoned_date,
        "avg_order_value": avg_order_value,
        "sessions_last_30d": sessions_last_30d,
        "num_cart_items": num_cart_items,
        "engagement_score": engagement_score,
        "profitability_score": profitability_score
    })
    df['days_since_abandon'] = (today - pd.to_datetime(df['cart_abandoned_date'])).dt.days
    return df

def build_mece_segments(df, aov_thresholds=(1000,3000), eng_thresholds=(0.4,0.7), prof_threshold=0.6, min_seg=MIN_SEG, max_seg=MAX_SEG):
    df = df.copy()
    lo, hi = aov_thresholds
    df['aov_bucket'] = df['avg_order_value'].apply(
        lambda a: "AOV Other" if pd.isna(a) else ("High AOV" if a > hi else "Mid AOV" if a > lo else "Low AOV"))
    e_lo, e_hi = eng_thresholds
    df['eng_bucket'] = df['engagement_score'].apply(
        lambda e: "Eng Other" if pd.isna(e) else ("High Eng" if e > e_hi else "Med Eng" if e > e_lo else "Low Eng"))
    df['prof_bucket'] = df['profitability_score'].apply(
        lambda p: "Prof Other" if pd.isna(p) else ("High Prof" if p > prof_threshold else "Low Prof"))
    df['segment_raw'] = df['aov_bucket'] + " | " + df['eng_bucket'] + " | " + df['prof_bucket']
    seg_counts = df['segment_raw'].value_counts().to_dict()
    final_map = {seg: seg for seg in seg_counts.keys()}
    def parent_of(seg, levels_up=1):
        parts = seg.split(" | ")
        return "ELSE" if levels_up >= len(parts) else " | ".join(parts[:-levels_up])
    changed, it = True, 0
    while changed and it < 10:
        it += 1
        changed = False
        mapped_sizes = {}
        for raw, mapped in final_map.items():
            mapped_sizes[mapped] = mapped_sizes.get(mapped, 0) + seg_counts.get(raw, 0)
        for raw, mapped in list(final_map.items()):
            if mapped_sizes.get(mapped, 0) < min_seg:
                parent = parent_of(mapped, 1)
                if parent != mapped:
                    final_map[raw] = parent
                    changed = True
    df['final_segment'] = df['segment_raw'].map(final_map)
    return df

def score_segments(df):
    df = df.copy()
    df['recency_factor'] = (7 - df['days_since_abandon'] + 1) / 8.0
    df['conversion_potential_ind'] = df['engagement_score'] * df['recency_factor']
    rng = np.random.default_rng(0)
    df['lift_ind'] = df.apply(lambda row: float(np.clip(
        rng.normal(loc=1.0 + 0.5*(row['engagement_score']-0.5) + 0.4*(row['profitability_score']-0.5), scale=0.12),
        0.5, 2.0)), axis=1)
    df['aov_pct'] = df['avg_order_value'].rank(pct=True)
    df['strategic_fit_ind'] = 0.6*df['aov_pct'] + 0.4*df['profitability_score']
    agg = df.groupby('final_segment').agg(
        size=('user_id','count'),
        conv_pot_mean=('conversion_potential_ind','mean'),
        lift_mean=('lift_ind','mean'),
        profit_mean=('profitability_score','mean'),
        strategic_mean=('strategic_fit_ind','mean'),
        avg_aov=('avg_order_value','mean')
    ).reset_index()
    agg['size_norm'] = (agg['size']-agg['size'].min())/(agg['size'].max()-agg['size'].min()) if agg['size'].max()!=agg['size'].min() else 0
    weights = {'conv_pot':0.32, 'lift':0.22, 'profitability':0.20, 'strategic':0.16, 'size_norm':0.10}
    agg['overall'] = (weights['conv_pot']*agg['conv_pot_mean'] +
                      weights['lift']*(agg['lift_mean']/2.0) +
                      weights['profitability']*agg['profit_mean'] +
                      weights['strategic']*agg['strategic_mean'] +
                      weights['size_norm']*agg['size_norm'])
    agg['valid'] = agg['size'].between(MIN_SEG, MAX_SEG)
    return agg

def tool_build_segments(args_json: str) -> str:
    args = json.loads(args_json)
    df_all = simulate_dataset(5000, TODAY)
    universe = df_all[df_all['cart_abandoned_date'].notna() & (df_all['days_since_abandon'] <= 7)].copy()
    segged = build_mece_segments(universe,
                                aov_thresholds=tuple(args.get('aov_thresholds',(1000,3000))),
                                eng_thresholds=tuple(args.get('eng_thresholds',(0.4,0.7))),
                                prof_threshold=args.get('prof_threshold',0.6),
                                min_seg=args.get('min_seg',MIN_SEG),
                                max_seg=args.get('max_seg',MAX_SEG))
    agg = score_segments(segged)
    agg.to_csv("agent_summary.csv", index=False)
    segged.to_csv("agent_segmented.csv", index=False)
    agg.to_json("agent_segments.json", orient='records')
    return json.dumps({"status":"ok","summary_csv":"agent_summary.csv","segmented_csv":"agent_segmented.csv","segments_count":len(agg)})


# Simple GUI (prompt to qury our agent)
# =====================
input_box = widgets.Textarea(
    placeholder='Type what you want the agent to do (e.g., "Segment users with AOV [1000,3000] and engagement [0.4,0.7]").',
    layout=widgets.Layout(width='100%', height='100px'),
    description='Prompt:'
)
output_box = widgets.Textarea(
    placeholder='Agent output here',
    layout=widgets.Layout(width='100%', height='200px'),
    description='Output:',
    disabled=True
)
button = widgets.Button(description='Run Agent', button_style='success', icon='play')

def on_click(b):
    clear_output(wait=True)
    display(input_box, button, output_box)
    # Fixed thresholds for demo
    args = {
        "aov_thresholds":[1000,3000],
        "eng_thresholds":[0.4,0.7],
        "prof_threshold":0.6,
        "min_seg":100,
        "max_seg":20000
    }
    #files that have the outputs from the agents query.
    result = tool_build_segments(json.dumps(args))
    output_box.value = f"Ran segmentation.\nFiles written:\n- agent_summary.csv\n- agent_segmented.csv\n- agent_segments.json\n\nResult JSON:\n{result}"

button.on_click(on_click)
display(input_box, button, output_box)
`

Textarea(value='I want to see which cart abandoners are high spenders versus bargain shoppers. Break them into…

Button(button_style='success', description='Run Agent', icon='play', style=ButtonStyle())

Textarea(value='', description='Output:', disabled=True, layout=Layout(height='200px', width='100%'), placehol…